<div class="alert alert-block alert-warning">
<b>Alert:</b> This QPU calculation costs $ ??? .
</div>

<div class="alert alert-block alert-success">
<b>This simulation runs a circuit with a varying number of gates for the purpose of detecting depolarization error. </b>
<div>Obs. 1: The data is saved in external files so that it can be worked on without incurring new costs.</div>
<div>Obs. 2: For new simulation parameters, save in a file with a different name.</div>
</div>

# Instructions if you are running outside AWS cloud

1. Install aws-cli:

`sudo apt install aws-cli`

2. Setup aws credentials:

`aws configure --profile "default"`

3. Create virtual environment 

`python -m venv /path/to/new/virtual/environment`

4. Activate virtual environment

`source /path/to/new/virtual/environment/bin/activate`

In [ ]:
# Install this modules on your virtual environment: an open source library for interacting
# with quantum computing devices and simulators on Amazon Braket
!pip install -r requirements

### AWS remote login

In [4]:
from boto3 import Session
from braket.aws import AwsSession

# Start Boto3 Session with your credentials profile
boto_session = Session(profile_name="default")

# Initialize Braket session with Boto3 Session credentials
aws_session = AwsSession(boto_session=boto_session)

# Importing modules

In [5]:
from braket.tracking import Tracker

t = Tracker().start()

In [6]:
# AWS imports: Import Braket SDK modules
import math

from braket.circuits import Circuit, Observable
from braket.devices import LocalSimulator

In [8]:
from braket.aws import AwsDevice, AwsQuantumTask
from braket.error_mitigation import Debias

# Introdução ao Método de Monte Carlo Quântico

O Método de Monte Carlo Quântico (QMC) é uma técnica numérica poderosa usada para resolver problemas complexos na física quântica. Ele combina a abordagem estatística do Monte Carlo com os fundamentos da mecânica quântica para calcular propriedades de sistemas quânticos interagentes ou não interagentes.

## Motivação
Sistemas quânticos com muitos corpos são difíceis de tratar analiticamente devido à alta dimensionalidade do espaço de estados e às interações entre partículas. Métodos de Monte Carlo fornecem uma maneira eficiente de explorar esse espaço de estados usando amostragem estatística.

O QMC é amplamente utilizado em diversas áreas da física, incluindo:
- Sistemas de muitos corpos (ex.: gases quânticos, líquidos de Fermi e bósons interagentes);
- Simulações de materiais;
- Física de partículas e teoria de campos.

## Fundamentos do Método
A ideia básica do QMC é usar métodos de Monte Carlo para resolver integrais de alta dimensão que aparecem frequentemente na mecânica quântica. No contexto de QMC, estas integrais geralmente estão associadas ao cálculo de valores esperados:

$$
\langle O \rangle = \frac{\int \psi^*(x) O \psi(x) \, dx}{\int \psi^*(x) \psi(x) \, dx},
$$

onde:
- \( O \) é o operador quântico (ex.: energia, posição);
- $\psi(x)$ é a função de onda do sistema.

Ao invés de calcular diretamente as integrais (o que é impraticável em muitas dimensões), o QMC utiliza **amostragem de Monte Carlo** para estimar esses valores.

## Técnicas Comuns de QMC
### 1. Monte Carlo Variacional (VMC)
O VMC usa uma função de onda parametrizada \( \psi_\theta(x) \) e busca minimizar a energia esperada:

$$
E[\psi_\theta] = \frac{\langle \psi_\theta | H | \psi_\theta \rangle}{\langle \psi_\theta | \psi_\theta \rangle}.
$$

Essa técnica é ideal para encontrar aproximações para o estado fundamental de sistemas quânticos.

### 2. Difusão Monte Carlo (DMC)
O DMC melhora os resultados do VMC ao simular diretamente a evolução temporal da função de onda no limite de tempo infinito. Ele fornece soluções mais precisas para o estado fundamental.

### 3. Método de Metropolis-Hastings
Esse é um método central no QMC, usado para gerar amostras no espaço de configurações com uma distribuição proporcional a $ |\psi(x)|^2 $. Ele permite que o QMC seja usado para amostrar estados quânticos de forma eficiente.

## Vantagens e Limitações
### Vantagens:
- Permite estudar sistemas quânticos altamente complexos e de alta dimensionalidade.
- Pode ser aplicado tanto a sistemas interagentes quanto a sistemas não interagentes.
- Reduz o custo computacional em comparação com métodos determinísticos.

### Limitações:
- Exige conhecimento prévio ou uma boa aproximação da função de onda \( \psi(x) \).
- Pode sofrer do problema de "sinal", especialmente em sistemas fermiônicos.

## Aplicações
- Simulação de gases de bósons e férmions.
- Estudo de transições de fase quânticas.
- Cálculo de propriedades de materiais em física do estado sólido.

---

No decorrer deste notebook, implementaremos o Método de Monte Carlo Quântico assumindo que a função de onda do sistema é uma **gaussiana**. Começaremos explorando a amostragem de $ |\psi(x)|^2 $ e calculando propriedades básicas do sistema.


# Define Simulation's Parameters Set

In [1]:
test_parameters = dict(n_qubits=1, n_shots=10, max_deph=100, step=2, isVerbose=True)
exec_parameters = dict(n_qubits=10, n_shots=2500, max_deph=600, step=100, isVerbose=False)

parameters_set = test_parameters

In [ ]:
import numpy as np

# Função de onda
def psi(x, alpha):
    return np.exp(-alpha * x**2)

# Densidade de probabilidade
def prob(x, alpha):
    return np.abs(psi(x, alpha))**2
